## Nodebooks: Introducing Node.js Data Science Notebooks

Notebooks (that’s Jupyter/IPython Notebooks, not Moleskine® notebooks) are where data scientists process, analyse, and visualise data in an iterative, collaborative environment. Like other developers, I am not a data scientist, but I do like the idea of having a scratchpad where I can write some code, iteratively work on some algorithms, and visualise the results quickly.
To that end, David Taieb and I created pixiedust_node, an add-on for Jupyter notebooks that allows Node.js/JavaScript to run inside notebook cells. It’s built on the popular PixieDust helper library. So let’s get started!


## Part 1: Variables, functions, and promises


### Installing
Install the [`pixiedust`](https://pypi.python.org/pypi/pixiedust) and [`pixiedust_node`](https://pypi.python.org/pypi/pixiedust-node) packages using `pip`, the Python package manager. 

In [ ]:
!pip install pixiedust
!pip install pixiedust_node --upgrade

### Using pixiedust_node
Now we can import `pixiedust_node` into our notebook:

In [ ]:
import pixiedust_node

And then we can write JavaScript code in cells whose first line is `%%node`:

In [ ]:
%%node
// get the current date
var date = new Date();

It’s that easy! We can have Python and Node.js in the same notebook. Cells are Python by default, but simply starting a cell with `%%node` indicates that the next lines will be JavaScript.

### Displaying HTML and images in notebook cells
We can use the `html` function to render HTML code in a cell:

In [ ]:
%%node
var str = '<h2>Quote</h2><blockquote cite="https://www.quora.com/Albert-Einstein-reportedly-said-The-true-sign-of-intelligence-is-not-knowledge-but-imagination-What-did-he-mean">"Imagination is more important than knowledge"\nAlbert Einstein</blockquote>';
html(str)

If we have an image we want to render, we can do that with the `image` function:

In [ ]:
%%node
var url = 'https://media.giphy.com/media/JIX9t2j0ZTN9S/giphy.gif';
image(url);

### Printing JavaScript variables
Calling the `print` function within your JavaScript code is the same as calling `print` in your Python code.

In [ ]:
%%node
var x = { a:1, b:'two', c: true };
print(x);

### Visualizing data using PixieDust
You can also use PixieDust’s `display` function to render data graphically.

In [ ]:
%%node
var data = [];
for (var i = 0; i < 1000; i++) {
    var x = 2*Math.PI * i/ 360;
    var obj = {
      x: x,
      i: i,
      sin: Math.sin(x),
      cos: Math.cos(x),
      tan: Math.tan(x)
    };
    data.push(obj);
}
// render data 
display(data);

<img src="images/display_sin_cos.png"></img>

PixieDust presents visualisations of data frames using Matplotlib, Bokeh, Brunel, d3, Google Maps and, MapBox. No code is required on your part because PixieDust presents simple pull-down menus and a friendly point-and-click interface, allowing you to configure how the data is presented:

<img src="images/pd_chart_types.png"></img>

### Adding npm modules
There are thousands of libraries and tools in the npm repository, Node.js’s package manager. It’s essential that we can install npm libraries and use them in our notebook code.
Let’s say we want to make some HTTP calls to an external API service. We could deal with Node.js’s low-level HTTP library, or an easier option would be to use the ubiquitous `request` npm module.
Once we have pixiedust_node set up, installing an npm module is as simple as running `npm.install` in a Python cell:

In [ ]:
npm.install('request');

Once installed, you may `require` the module in your JavaScript code:

In [ ]:
%%node
var request = require('request');
var r = {
    method:'GET',
    url: 'http://api.open-notify.org/iss-now.json',
    json: true
};
request(r, function(err, req, body) {
    print(body);
});


As an HTTP request is an asynchronous action, the `request` library calls our callback function when the operation has completed. Inside that function, we can call print to render the data.
We can organise our code into functions to encapsulate complexity and make it easier to reuse code. We can create a function to get the current position of the International Space Station in one notebook cell:

In [ ]:
%%node
var request = require('request');
var getPosition = function(callback) {
    var r = {
        method:'GET',
        url: 'http://api.open-notify.org/iss-now.json',
        json: true
    };
    request(r, function(err, req, body) {
        var obj = null;
        if (!err) {
            obj = body.iss_position
            obj.latitude = parseFloat(obj.latitude);
            obj.longitude = parseFloat(obj.longitude);
            obj.time = new Date().getTime();       
        }
        callback(err, obj);
    });
};

And use it in another cell:

In [ ]:
%%node
getPosition(function(err, data) {
    print(data);
});

### Promises
If you prefer to work with JavaScript Promises when writing asynchronous code, then that’s okay too. Let’s rewrite our `getPosition` function to return a Promise. First we're going to install the `request-promise` module from npm:

In [ ]:
npm.install( ('request', 'request-promise') )

Notice how you can install multiple modules in a single call. Just pass in a Python `list` or `tuple`.
Then we can refactor our function a little:

In [ ]:
%%node
var request = require('request-promise');
var getPosition = function(callback) {
    var r = {
        method:'GET',
        url: 'http://api.open-notify.org/iss-now.json',
        json: true
    };
    return request(r).then(function(body) {
        var obj = null;
        obj = body.iss_position;
        obj.latitude = parseFloat(obj.latitude);
        obj.longitude = parseFloat(obj.longitude);
        obj.time = new Date().getTime();         
        return obj;
    });
};

And call it in the Promises style:

In [ ]:
%%node
getPosition().then(function(data) {
  print(data);
}).catch(function(err) {
  print(err);    
});

Or call it in a more compact form:

In [ ]:
%%node
getPosition().then(print).catch(print);


In the next part of this notebook we'll illustrate how you can access local and remote data sources from within the notebook.

***
# Part 2: Working with data sources

You can access any data source using your favorite public or home-grown packages. In the second part of this notebook you'll learn how to retrieve data from an Apache CouchDB (or Cloudant) database or a Redis database and visualize it using PixieDust or third-party libraries. 

## 2.1 Accessing Claudant data sources


To access data stored in Apache CouchDB or Cloudant database, we can use the [`cloudant-quickstart`](https://www.npmjs.com/package/cloudant-quickstart) npm module:

In [ ]:
npm.install('cloudant-quickstart')

With our Cloudant URL, we can start exploring the data in Node.js. First we make a connection to the remote Cloudant database:

In [ ]:
%%node
// connect to Cloudant using cloudant-quickstart
var cqs = require('cloudant-quickstart');
var cities = cqs('https://reader.cloudant.com/cities');

Now we have an object cities that we can use to access the database. 

### Exploring the data using Node.js in a notebook 

If we know the IDs of documents, we can retrieve them singly:

In [ ]:
%%node
cities.get('2636749').then(print).catch(print);

Or in bulk:

In [ ]:
%%node
cities.get(['4562407','2636749','3530597']).then(print).catch(print);

Instead of just calling `print` to output the JSON, we can bring PixieDust's `display` function to bear by passing it an array of data to visualize:

In [ ]:
%%node
cities.get(['4562407','2636749','3530597']).then(display).catch(print);

We can also query a subset of the data using the `query` function, passing it a Cloudant Query statement:

In [ ]:
%%node
// fetch cities in UK above latitude 54 degrees north
cities.query({country:'GB', latitude: { "$gt": 54}}).then(display).catch(print);

<img src="images/mapbox_uk.png"></img>

### Aggregating data
The `cloudant-quickstart` library also allows aggregations (sum, count, stats) to be performed in the Cloudant database.
Let’s calculate the sum of the population field:

In [ ]:
%%node
cities.sum('population').then(print).catch(print);

Or compute the sum of the `population`, grouped by the `country` field:

In [ ]:
%%node
cities.sum('population','country').then(print).catch(print);

The `cloudant-quickstart` package is just one of several Node.js libraries that you can use to access Apache CouchDB or Cloudant. Follow [this link](https://medium.com/ibm-watson-data-lab/choosing-a-cloudant-library-d14c06f3d714) to learn more about your options. 

### Visualizing data using custom charts

If you prefer, you can also use third-party charting packages to visualize your data, such as [`quiche`](https://www.npmjs.com/package/quiche).

In [ ]:
npm.install('quiche');

In [ ]:
%%node
var Quiche = require('quiche');
var pie = new Quiche('pie');

// fetch cities in UK
cities.query({name: 'Cambridge'}).then(function(data) {

  var colors = ['ff00ff','0055ff', 'ff0000', 'ffff00', '00ff00','0000ff'];
  for(i in data) {
    var city = data[i];
    pie.addData(city.population, city.name + '(' + city.country +')', colors[i]);
  }
  var imageUrl = pie.getUrl(true);
  image(imageUrl);    
});

## 2.2 Accessing Redis data sources
... to be continued

***
## Part 3: Sharing data between Python and Node.js cells

to be continued ...

#### References:
 * [Nodebooks: Introducing Node.js Data Science Notebooks](https://medium.com/ibm-watson-data-lab/nodebooks-node-js-data-science-notebooks-aa140bea21ba)
 * [Nodebooks: Sharing Data Between Node.js & Python](https://medium.com/ibm-watson-data-lab/nodebooks-sharing-data-between-node-js-python-3a4acae27a02)
 * [Sharing Variables Between Python & Node.js in Jupyter Notebooks](https://medium.com/ibm-watson-data-lab/sharing-variables-between-python-node-js-in-jupyter-notebooks-682a79d4bdd9)